In [35]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [36]:
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
choices = ['UUU','UCA','UAC','UGG','CUC','CCG','CAA','CGU','AUA','ACU','AAG','GUG','GCC','GAU','GGA']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in choices:
            blockArray[i][key] = '*'
reductionist15 = {}
reductionist16 = {}
reductionist17 = {}
reductionist18 = {}
reductionist19 = {}
reductionist20 = {}
for i in range (len(blockArray)):
    reductionist15.update(blockArray[i])
reductionist16.update(reductionist15)
reductionist16['AAU'] = 'N'
reductionist17.update(reductionist16)  
reductionist17['GAU'] = 'D'
reductionist18.update(reductionist17) 
reductionist18['AUG'] = 'M'
reductionist19.update(reductionist18) 
reductionist19['CAU'] ='H'
reductionist20.update(reductionist19)
reductionist20['UGC'] = 'C'

In [37]:
reductionist20Table = codonTable(reductionist20)
reductionist19Table = codonTable(reductionist19)
reductionist18Table = codonTable(reductionist18)
reductionist17Table = codonTable(reductionist17)
reductionist16Table = codonTable(reductionist16)
reductionist15Table = codonTable(reductionist15)

In [38]:
reductionist15Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : Q  CGA : *
  G  CUG : *  CCG : P  CAG : *  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : *
  A  AUA : I  ACA : *  AAA : *  AGA : *
  G  AUG : *  ACG : *  AAG : K  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : V  GCG : *  GAG : *  GGG : *

In [39]:
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstEight = ['UUU','UGG','CAG','CGU','AUG','ACU','GCG','GAU']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstEight:
            blockArray[i][key] = '*'
greenTable8 = {}
greenTable9 = {}
greenTable10 = {}
greenTable11 = {}
greenTable12 = {}
greenTable13 = {}
greenTable14 = {}            
greenTable15 = {}
greenTable16 = {}
greenTable17 = {}
greenTable18 = {}
greenTable19 = {}
greenTable20 = {}
for i in range (len(blockArray)):
    greenTable8.update(blockArray[i])
greenTable9.update(greenTable8)
greenTable9['AAA'] = 'K'
greenTable10.update(greenTable9)
greenTable10['GGA'] = 'G'
greenTable11.update(greenTable10)  
greenTable11['UCA'] = 'S'
greenTable12.update(greenTable11) 
greenTable12['CCC'] = 'P'
greenTable13.update(greenTable12) 
greenTable13['CUA'] ='L'
greenTable14.update(greenTable13)
greenTable14['UAC'] = 'Y'
greenTable15.update(greenTable14)
greenTable15['GUC'] = 'V'
greenTable16.update(greenTable15)
greenTable16['CAU'] = 'H'
greenTable17.update(greenTable16)  
greenTable17['GAG'] = 'E'
greenTable18.update(greenTable17) 
greenTable18['AUU'] = 'I'
greenTable19.update(greenTable18) 
greenTable19['AAC'] ='N'
greenTable20.update(greenTable19)
greenTable20['UGU'] = 'C'

In [40]:
green20Table = codonTable(greenTable20)
green19Table = codonTable(greenTable19)
green18Table = codonTable(greenTable18)
green17Table = codonTable(greenTable17)
green16Table = codonTable(greenTable16)
green15Table = codonTable(greenTable15)
green14Table = codonTable(greenTable14)
green13Table = codonTable(greenTable13)
green12Table = codonTable(greenTable12)
green11Table = codonTable(greenTable11)
green10Table = codonTable(greenTable10)
green9Table = codonTable(greenTable9)
green8Table = codonTable(greenTable8)

In [41]:
strainGreen8 = strain(table =green8Table)
strainGreen8.mu
strainGreen9 = strain(table =green9Table)
strainGreen9.mu
strainGreen10 = strain(table =green10Table)
strainGreen10.mu
strainGreen11 = strain(table =green11Table)
strainGreen11.mu
strainGreen12 = strain(table =green12Table)
strainGreen12.mu
strainGreen13 = strain(table =green13Table)
strainGreen13.mu
strainGreen14 = strain(table =green14Table)
strainGreen14.mu
strainGreen15 = strain(table =green15Table)
strainGreen15.mu
strainGreen16 = strain(table =green16Table)
strainGreen16.mu
strainGreen17 = strain(table =green17Table)
strainGreen17.mu
strainGreen18 = strain(table =green18Table)
strainGreen18.mu
strainGreen19 = strain(table =green19Table)
strainGreen19.mu
strainGreen20 = strain(table =green20Table)
strainGreen20.mu
greenMutationRate = [strainGreen8.mu, strainGreen9.mu, strainGreen10.mu, strainGreen11.mu, strainGreen12.mu, 
                     strainGreen13.mu, strainGreen14.mu, strainGreen15.mu, strainGreen16.mu, strainGreen17.mu, 
                     strainGreen18.mu, strainGreen19.mu, strainGreen20.mu]

In [42]:
greenMutationRate

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0612244897959183e-07,
 6.122448979591837e-07,
 9.183673469387756e-07,
 1.122448979591837e-06,
 1.4285714285714286e-06]

In [43]:
green8Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : *  UGC : *
  A  UUA : *  UCA : *  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : *  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : *  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : *  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : *
  G  GUG : *  GCG : A  GAG : *  GGG : *

In [44]:
green15Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : *  CGU : R
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : *  GGG : *

In [45]:
green20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : C
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : R
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : *  CAG : Q  CGG : *
A U  AUU : I  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : N  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : E  GGG : *

In [46]:
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstEight = ['UUU','UGG','AUG','CCG','CAU','ACU','GAG','GGU']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstEight:
            blockArray[i][key] = '*'
redTable8 = {}
redTable9 = {}
redTable10 = {}
redTable11 = {}
redTable12 = {}
redTable13 = {}
redTable14 = {}            
redTable15 = {}
redTable16 = {}
redTable17 = {}
redTable18 = {}
redTable19 = {}
redTable20 = {}
for i in range (len(blockArray)):
    redTable8.update(blockArray[i])
redTable9.update(redTable8)
redTable9['AAC'] = 'A'
redTable10.update(redTable9)
redTable10['AGA'] = 'R'
redTable11.update(redTable10)  
redTable11['UCA'] = 'S'
redTable12.update(redTable11) 
redTable12['GCC'] = 'A'
redTable13.update(redTable12) 
redTable13['CUC'] ='L'
redTable14.update(redTable13)
redTable14['UAU'] = 'Y'
redTable15.update(redTable14)
redTable15['GUA'] = 'V'
redTable16.update(redTable15)
redTable16['AAG'] = 'K'
redTable17.update(redTable16)  
redTable17['GAU'] = 'D'
redTable18.update(redTable17) 
redTable18['CAA'] = 'Q'
redTable19.update(redTable18) 
redTable19['AUU'] ='I'
redTable20.update(redTable19)
redTable20['UGC'] = 'C'

In [47]:
red20Table = codonTable(redTable20)
red19Table = codonTable(redTable19)
red18Table = codonTable(redTable18)
red17Table = codonTable(redTable17)
red16Table = codonTable(redTable16)
red15Table = codonTable(redTable15)
red14Table = codonTable(redTable14)
red13Table = codonTable(redTable13)
red12Table = codonTable(redTable12)
red11Table = codonTable(redTable11)
red10Table = codonTable(redTable10)
red9Table = codonTable(redTable9)
red8Table = codonTable(redTable8)

In [34]:
red15Table.codonTable

U        C        A        G
U U  UUU : F  UCU : *  UAU : Y  UGU : *
  C  UUC : *  UCC : *  UAC : *  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : *  CGG : *
A U  AUU : *  ACU : T  AAU : *  AGU : *
  C  AUC : *  ACC : *  AAC : A  AGC : *
  A  AUA : *  ACA : *  AAA : *  AGA : R
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : *  GAU : *  GGU : G
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : *  GGA : *
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [48]:
strainRed8 = strain(table =red8Table)
strainRed8.mu
strainRed9 = strain(table =red9Table)
strainRed9.mu
strainRed10 = strain(table =red10Table)
strainRed10.mu
strainRed11 = strain(table =red11Table)
strainRed11.mu
strainRed12 = strain(table =red12Table)
strainRed12.mu
strainRed13 = strain(table =red13Table)
strainRed13.mu
strainRed14 = strain(table =red14Table)
strainRed14.mu
strainRed15 = strain(table =red15Table)
strainRed15.mu
strainRed16 = strain(table =red16Table)
strainRed16.mu
strainRed17 = strain(table =red17Table)
strainRed17.mu
strainRed18 = strain(table =red18Table)
strainRed18.mu
strainRed19 = strain(table =red19Table)
strainRed19.mu
strainRed20 = strain(table =red20Table)
strainRed20.mu
redMutationRate = [strainRed8.mu, strainRed9.mu, strainRed10.mu, strainRed11.mu, strainRed12.mu, 
                     strainRed13.mu, strainRed14.mu, strainRed15.mu, strainRed16.mu, strainRed17.mu, 
                     strainRed18.mu, strainRed19.mu, strainRed20.mu]

In [49]:
redMutationRate

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0408163265306121e-07,
 2.0408163265306121e-07,
 5.102040816326531e-07,
 9.183673469387756e-07,
 1.0204081632653063e-06,
 1.3265306122448982e-06,
 1.4285714285714286e-06]

In [50]:
utils.promiscuity(redTable15)

{'AAA': '*',
 'AAC': 'A',
 'AAG': '*',
 'AAU': 'A',
 'ACA': '*',
 'ACC': '*',
 'ACG': '*',
 'ACU': 'T',
 'AGA': 'R',
 'AGC': '*',
 'AGG': 'R',
 'AGU': '*',
 'AUA': '*',
 'AUC': '*',
 'AUG': 'M',
 'AUU': '*',
 'CAA': '*',
 'CAC': '*',
 'CAG': '*',
 'CAU': 'H',
 'CCA': '*',
 'CCC': '*',
 'CCG': 'P',
 'CCU': '*',
 'CGA': '*',
 'CGC': '*',
 'CGG': '*',
 'CGU': '*',
 'CUA': '*',
 'CUC': 'L',
 'CUG': '*',
 'CUU': 'L',
 'GAA': '*',
 'GAC': '*',
 'GAG': 'E',
 'GAU': '*',
 'GCA': '*',
 'GCC': 'A',
 'GCG': '*',
 'GCU': 'A',
 'GGA': '*',
 'GGC': '*',
 'GGG': '*',
 'GGU': 'G',
 'GUA': 'V',
 'GUC': '*',
 'GUG': 'V',
 'GUU': '*',
 'UAA': '*',
 'UAC': '*',
 'UAG': '*',
 'UAU': 'Y',
 'UCA': 'S',
 'UCC': '*',
 'UCG': 'S',
 'UCU': '*',
 'UGA': '*',
 'UGC': '*',
 'UGG': 'W',
 'UGU': '*',
 'UUA': '*',
 'UUC': '*',
 'UUG': '*',
 'UUU': 'F'}

In [51]:
promiscuousRed8 =  utils.promiscuity(redTable8, allow_ambiguous = True)
promiscuousRed9 =  utils.promiscuity(redTable9, allow_ambiguous = True)
promiscuousRed10 =  utils.promiscuity(redTable10, allow_ambiguous = True)
promiscuousRed11 =  utils.promiscuity(redTable11, allow_ambiguous = True)
promiscuousRed12 =  utils.promiscuity(redTable12, allow_ambiguous = True)
promiscuousRed13 =  utils.promiscuity(redTable13, allow_ambiguous = True)
promiscuousRed14 =  utils.promiscuity(redTable14, allow_ambiguous = True)
promiscuousRed15 =  utils.promiscuity(redTable15, allow_ambiguous = True)
promiscuousRed16 =  utils.promiscuity(redTable16, allow_ambiguous = True)
promiscuousRed17 =  utils.promiscuity(redTable17, allow_ambiguous = True)
promiscuousRed18 =  utils.promiscuity(redTable18, allow_ambiguous = True)
promiscuousRed19 =  utils.promiscuity(redTable19, allow_ambiguous = True)
promiscuousRed20 =  utils.promiscuity(redTable20, allow_ambiguous = True)

In [52]:
promiscuousGreen8 =  utils.promiscuity(greenTable8, allow_ambiguous = True)
promiscuousGreen9 =  utils.promiscuity(greenTable9, allow_ambiguous = True)
promiscuousGreen10 =  utils.promiscuity(greenTable10, allow_ambiguous = True)
promiscuousGreen11 =  utils.promiscuity(greenTable11, allow_ambiguous = True)
promiscuousGreen12 =  utils.promiscuity(greenTable12, allow_ambiguous = True)
promiscuousGreen13 =  utils.promiscuity(greenTable13, allow_ambiguous = True)
promiscuousGreen14 =  utils.promiscuity(greenTable14, allow_ambiguous = True)
promiscuousGreen15 =  utils.promiscuity(greenTable15, allow_ambiguous = True)
promiscuousGreen16 =  utils.promiscuity(greenTable16, allow_ambiguous = True)
promiscuousGreen17 =  utils.promiscuity(greenTable17, allow_ambiguous = True)
promiscuousGreen18 =  utils.promiscuity(greenTable18, allow_ambiguous = True)
promiscuousGreen19 =  utils.promiscuity(greenTable19, allow_ambiguous = True)
promiscuousGreen20 =  utils.promiscuity(greenTable20, allow_ambiguous = True)

In [53]:
promiscuousRed8Table = codonTable(promiscuousRed8)
promiscuousRed9Table = codonTable(promiscuousRed9)
promiscuousRed10Table = codonTable(promiscuousRed10)
promiscuousRed11Table = codonTable(promiscuousRed11)
promiscuousRed12Table = codonTable(promiscuousRed12)
promiscuousRed13Table = codonTable(promiscuousRed13)
promiscuousRed14Table = codonTable(promiscuousRed14)
promiscuousRed15Table = codonTable(promiscuousRed15)
promiscuousRed16Table = codonTable(promiscuousRed16)
promiscuousRed17Table = codonTable(promiscuousRed17)
promiscuousRed18Table = codonTable(promiscuousRed18)
promiscuousRed19Table = codonTable(promiscuousRed19)
promiscuousRed20Table = codonTable(promiscuousRed20)

In [54]:
promiscuousGreen8Table = codonTable(promiscuousGreen8)
promiscuousGreen9Table = codonTable(promiscuousGreen9)
promiscuousGreen10Table = codonTable(promiscuousGreen10)
promiscuousGreen11Table = codonTable(promiscuousGreen11)
promiscuousGreen12Table = codonTable(promiscuousGreen12)
promiscuousGreen13Table = codonTable(promiscuousGreen13)
promiscuousGreen14Table = codonTable(promiscuousGreen14)
promiscuousGreen15Table = codonTable(promiscuousGreen15)
promiscuousGreen16Table = codonTable(promiscuousGreen16)
promiscuousGreen17Table = codonTable(promiscuousGreen17)
promiscuousGreen18Table = codonTable(promiscuousGreen18)
promiscuousGreen19Table = codonTable(promiscuousGreen19)
promiscuousGreen20Table = codonTable(promiscuousGreen20)

In [55]:
promiscuousStrainRed8 = strain(table =promiscuousRed8Table)
promiscuousStrainRed8.mu
promiscuousStrainRed9 = strain(table =promiscuousRed9Table)
promiscuousStrainRed9.mu
promiscuousStrainRed10 = strain(table =promiscuousRed10Table)
promiscuousStrainRed10.mu
promiscuousStrainRed11 = strain(table =promiscuousRed11Table)
promiscuousStrainRed11.mu
promiscuousStrainRed12 = strain(table =promiscuousRed12Table)
promiscuousStrainRed12.mu
promiscuousStrainRed13 = strain(table =promiscuousRed13Table)
promiscuousStrainRed13.mu
promiscuousStrainRed14 = strain(table =promiscuousRed14Table)
promiscuousStrainRed14.mu
promiscuousStrainRed15 = strain(table =promiscuousRed15Table)
promiscuousStrainRed15.mu
promiscuousStrainRed16 = strain(table =promiscuousRed16Table)
promiscuousStrainRed16.mu
promiscuousStrainRed17 = strain(table =promiscuousRed17Table)
promiscuousStrainRed17.mu
promiscuousStrainRed18 = strain(table =promiscuousRed18Table)
promiscuousStrainRed18.mu
promiscuousStrainRed19 = strain(table =promiscuousRed19Table)
promiscuousStrainRed19.mu
promiscuousStrainRed20 = strain(table =promiscuousRed20Table)
promiscuousStrainRed20.mu
promiscuousRedMutationRate = [promiscuousStrainRed8.mu, promiscuousStrainRed9.mu, promiscuousStrainRed10.mu, 
                   promiscuousStrainRed11.mu, promiscuousStrainRed12.mu, promiscuousStrainRed13.mu, 
                   promiscuousStrainRed14.mu, promiscuousStrainRed15.mu, promiscuousStrainRed16.mu,
                   promiscuousStrainRed17.mu, promiscuousStrainRed18.mu, promiscuousStrainRed19.mu, 
                   promiscuousStrainRed20.mu]

In [56]:
promiscuousRedMutationRate

[0.0,
 2.0408163265306121e-07,
 4.0816326530612243e-07,
 6.122448979591837e-07,
 8.163265306122449e-07,
 1.0204081632653063e-06,
 1.3265306122448982e-06,
 1.5306122448979593e-06,
 2.0408163265306125e-06,
 2.6530612244897964e-06,
 3.1632653061224496e-06,
 3.6734693877551024e-06,
 4.183673469387755e-06]

In [57]:
promiscuousStrainGreen8 = strain(table =promiscuousGreen8Table)
promiscuousStrainGreen8.mu
promiscuousStrainGreen9 = strain(table =promiscuousGreen9Table)
promiscuousStrainGreen9.mu
promiscuousStrainGreen10 = strain(table =promiscuousGreen10Table)
promiscuousStrainGreen10.mu
promiscuousStrainGreen11 = strain(table =promiscuousGreen11Table)
promiscuousStrainGreen11.mu
promiscuousStrainGreen12 = strain(table =promiscuousGreen12Table)
promiscuousStrainGreen12.mu
promiscuousStrainGreen13 = strain(table =promiscuousGreen13Table)
promiscuousStrainGreen13.mu
promiscuousStrainGreen14 = strain(table =promiscuousGreen14Table)
promiscuousStrainGreen14.mu
promiscuousStrainGreen15 = strain(table =promiscuousGreen15Table)
promiscuousStrainGreen15.mu
promiscuousStrainGreen16 = strain(table =promiscuousGreen16Table)
promiscuousStrainGreen16.mu
promiscuousStrainGreen17 = strain(table =promiscuousGreen17Table)
promiscuousStrainGreen17.mu
promiscuousStrainGreen18 = strain(table =promiscuousGreen18Table)
promiscuousStrainGreen18.mu
promiscuousStrainGreen19 = strain(table =promiscuousGreen19Table)
promiscuousStrainGreen19.mu
promiscuousStrainGreen20 = strain(table =promiscuousGreen20Table)
promiscuousStrainGreen20.mu
promiscuousGreenMutationRate = [promiscuousStrainGreen8.mu, promiscuousStrainGreen9.mu, promiscuousStrainGreen10.mu, 
                   promiscuousStrainGreen11.mu, promiscuousStrainGreen12.mu, promiscuousStrainGreen13.mu, 
                   promiscuousStrainGreen14.mu, promiscuousStrainGreen15.mu, promiscuousStrainGreen16.mu,
                   promiscuousStrainGreen17.mu, promiscuousStrainGreen18.mu, promiscuousStrainGreen19.mu, 
                   promiscuousStrainGreen20.mu]

In [58]:
promiscuousGreenMutationRate

[0.0,
 2.0408163265306121e-07,
 4.0816326530612243e-07,
 6.122448979591837e-07,
 8.163265306122449e-07,
 1.0204081632653063e-06,
 1.2244897959183673e-06,
 1.4285714285714286e-06,
 1.938775510204082e-06,
 2.4489795918367347e-06,
 2.8571428571428573e-06,
 3.877551020408164e-06,
 4.2857142857142855e-06]

In [59]:
greenBenMutRate = np.array(greenMutationRate)
redBenMutRate = np.array(redMutationRate)
compMutRateGminusR = np.array(greenBenMutRate-redBenMutRate)
compMutRateRminusG = np.array(redBenMutRate-greenBenMutRate)

In [60]:
compMutRateGminusR

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.04081633e-07, -2.04081633e-07,
       -2.04081633e-07, -3.06122449e-07, -1.02040816e-07, -2.04081633e-07,
        0.00000000e+00])

In [61]:
compMutRateRminusG

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.04081633e-07, 2.04081633e-07,
       2.04081633e-07, 3.06122449e-07, 1.02040816e-07, 2.04081633e-07,
       0.00000000e+00])

In [62]:
promiscuousGreenBenMutRate = np.array(promiscuousGreenMutationRate)
promiscuousRedBenMutRate = np.array(promiscuousRedMutationRate)
promiscuousCompMutRateGminusR = np.array(promiscuousGreenBenMutRate-promiscuousRedBenMutRate)
promiscuousCompMutRateRminusG = np.array(promiscuousRedBenMutRate-promiscuousGreenBenMutRate)

In [63]:
promiscuousCompMutRateGminusR

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.02040816e-07, -1.02040816e-07,
       -1.02040816e-07, -2.04081633e-07, -3.06122449e-07,  2.04081633e-07,
        1.02040816e-07])

In [64]:
promiscuousCompMutRateRminusG

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.02040816e-07,  1.02040816e-07,
        1.02040816e-07,  2.04081633e-07,  3.06122449e-07, -2.04081633e-07,
       -1.02040816e-07])

In [65]:
promiscuousRed20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : Y  UGU : C
  C  UUC : *  UCC : *  UAC : *  UGC : C
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : S  UAG : *  UGG : W
C U  CUU : L  CCU : *  CAU : H  CGU : *
  C  CUC : L  CCC : *  CAC : *  CGC : *
  A  CUA : *  CCA : *  CAA : Q  CGA : *
  G  CUG : *  CCG : P  CAG : Q  CGG : *
A U  AUU : I  ACU : T  AAU : A  AGU : *
  C  AUC : *  ACC : *  AAC : A  AGC : *
  A  AUA : *  ACA : *  AAA : *  AGA : R
  G  AUG : M  ACG : *  AAG : K  AGG : R
G U  GUU : *  GCU : A  GAU : D  GGU : G
  C  GUC : *  GCC : A  GAC : *  GGC : *
  A  GUA : V  GCA : *  GAA : *  GGA : *
  G  GUG : V  GCG : *  GAG : E  GGG : *

In [66]:
promiscuousGreen20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : Y  UGU : C
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : S  UAG : *  UGG : W
C U  CUU : *  CCU : P  CAU : H  CGU : R
  C  CUC : *  CCC : P  CAC : *  CGC : *
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : L  CCG : *  CAG : Q  CGG : *
A U  AUU : I  ACU : T  AAU : N  AGU : *
  C  AUC : *  ACC : *  AAC : N  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : K  AGG : *
G U  GUU : V  GCU : *  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : A  GAG : E  GGG : G

In [67]:
path = '/home/jonathan/Lab/Fast Fail/res/Reductionist Code/'
with open(path+'reductionist20.pickle', 'wb') as handle:
    pickle.dump(greenTable20, handle)
with open(path+'reductionist19.pickle', 'wb') as handle:
    pickle.dump(greenTable19, handle)
with open(path+'reductionist18.pickle', 'wb') as handle:
    pickle.dump(greenTable18, handle)
with open(path+'reductionist17.pickle', 'wb') as handle:
    pickle.dump(greenTable17, handle)
with open(path+'reductionist16.pickle', 'wb') as handle:
    pickle.dump(greenTable16, handle)
with open(path+'reductionist15.pickle', 'wb') as handle:
    pickle.dump(greenTable20, handle)